In [1]:
import sys
import numpy as np
import torch
import learn2learn as l2l
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms

from PIL.Image import LANCZOS
from csv import writer

from PIL import Image
import pandas as pd

In [2]:
torch.cuda.is_available()

False

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.7.1002181843/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.7.1002181843/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python39.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
# from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder
# from src.zoo.trident_utils import setup, inner_adapt_trident, loss, accuracy
# from data.loaders import CUBirds200
from src.zoo.trident_utils import inner_adapt_trident, setup


In [5]:
device = 'cpu'
n_ways = 5
k_shots = 1
q_shots = 10
order = False
inner_lr = 0.001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 5
meta_batch_size = 10
download = False

In [6]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'tiered'
        self.root = '../../dataset/mini_imagenet/'
        self.task_adapt = True
        self.task_adapt_fn = 'tafe'
        self.n_ways = 5
        self.k_shots = 1
        self.q_shots = 10
        self.zl = 64
        self.zs = 64
        self.reconstr = 'std'
        self.device = 'cpu'
        self.wn_channels = 32
        self.wm_channels = 64

args = Args(100, False, 0.01, 1, 1)

In [11]:
def testing(args, l):
    print('chaning now')
    args.dataset = 'cub'
    args.dataset = 'tiered'
    return l

In [12]:
a = testing(args, [1,2,3])
args.dataset

chaning now


'tiered'

In [ ]:
m = nn.LeakyReLU(0.2)
input = torch.randn(2)
output = m(input)
output

In [ ]:
m = nn.LeakyReLU(0.2)
#m = m(0.2)
input = torch.randn(2)
output = m(input)
output

In [ ]:
model = CEncoder(3, 32, args=args)
input = ttask[0]
output = model(input)
output

In [ ]:
learner = torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500.pt')

In [ ]:
learner.args

In [ ]:
learner.modules

In [ ]:
learner.state_dict()

In [ ]:
torch.save(learner.state_dict(), '../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt')

In [ ]:
learner = CCVAE(in_channels=3, base_channels=32,
                        n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
#opt = optim.Adam(learner.parameters(), 0.00015)

In [ ]:
train_tasks, valid_tasks, _, learner = setup(
    args.dataset, args.root, args.n_ways, args.k_shots, args.q_shots, order, inner_lr, args.device, download=download, task_adapt=args.task_adapt, args=args)
opt = optim.Adam(learner.parameters(), meta_lr)
reconst_loss = nn.MSELoss(reduction='none')


In [ ]:
ttask = train_tasks.sample()
model = learner.clone()

In [ ]:
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

In [ ]:
evaluation_loss

In [ ]:
evaluation_loss['elbo'].backward()

In [ ]:
torch.save(learner.state_dict(), './temp.pt')

In [ ]:
learner1 = CCVAE(in_channels=3, base_channels=32, n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
learner1 = l2l.algorithms.MAML(learner1, first_order=order, lr=inner_lr, allow_nograd=False)
learner1.load_state_dict(torch.load('../logs/DELPO_miniimagenet_5-way_5-shot_10-queries/og/model_28500.pt'))

In [ ]:
learner1.state_dict()

In [7]:
learner = CCVAE(in_channels=3, base_channels=32, n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
learner = l2l.algorithms.MAML(learner, first_order=order, lr=inner_lr, allow_nograd=False)
learner.load_state_dict(torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt'))

<All keys matched successfully>

In [8]:
learner

MAML(
  (module): CCVAE(
    (encoder): CEncoder(
      (net): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [ ]:
learner.state_dict()

In [ ]:
_, _, test_tasks, learner = setup(
    args.dataset, args.root, args.n_ways, args.k_shots, args.q_shots, order, inner_lr, args.device, download=download, task_adapt=args.task_adapt, args=args)

learner.load_state_dict(torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt'))
learner = learner.to(args.device)

In [ ]:
learner.load_state_dict(torch.load('../logs/DELPO_miniimagenet_5-way_5-shot_10-queries/og/model_28500.pt'))
learner = learner.to(args.device)

In [ ]:
learner.state_dict()

In [ ]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 10
order = False
inner_lr = 0.00014
meta_lr = 0.000014
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 10
meta_batch_size = 10
download = False

In [ ]:
ttask = test_tasks.sample()

In [ ]:
ttask

In [ ]:
model = learner.clone()
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

In [ ]:
evaluation_accuracy

In [ ]:
evaluation_loss